In [16]:
import os
import requests
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import AzureOpenAIEmbeddings

load_dotenv()
so_api_base_url = "https://api.stackexchange.com/2.3/search/advanced"
parameters = (
        f"?pagesize=100&page={1}&order=desc&sort=creation&answers=1&tagged={'neo4j'}"
        "&site=stackoverflow&filter=!*236eb_eL9rai)MOSNZ-6D3Q6ZKb0buI*IVotWaTb"
    )
data = requests.get(so_api_base_url + parameters).json()

In [17]:
def create_vector_index(driver, dimension: int) -> None:
    index_query = "CALL db.index.vector.createNodeIndex('stackoverflow', 'Question', 'embedding', $dimension, 'cosine')"
    try:
        driver.query(index_query, {"dimension": dimension})
    except:  # Already exists
        pass
    index_query = "CALL db.index.vector.createNodeIndex('top_answers', 'Answer', 'embedding', $dimension, 'cosine')"
    try:
        driver.query(index_query, {"dimension": dimension})
    except:  # Already exists
        pass

def create_constraints(driver):
    driver.query(
        "CREATE CONSTRAINT question_id IF NOT EXISTS FOR (q:Question) REQUIRE (q.id) IS UNIQUE"
    )
    driver.query(
        "CREATE CONSTRAINT answer_id IF NOT EXISTS FOR (a:Answer) REQUIRE (a.id) IS UNIQUE"
    )
    driver.query(
        "CREATE CONSTRAINT user_id IF NOT EXISTS FOR (u:User) REQUIRE (u.id) IS UNIQUE"
    )
    driver.query(
        "CREATE CONSTRAINT tag_name IF NOT EXISTS FOR (t:Tag) REQUIRE (t.name) IS UNIQUE"
    )

In [19]:
url="bolt://localhost:7687"
username="neo4j"
password="Password@123" # your password might be different

os.environ["NEO4J_URI"] = url
os.environ["NEO4J_USERNAME"] = username
os.environ["NEO4J_PASSWORD"]=password


# we are using openai embedding model
embedding_model_name = AzureOpenAIEmbeddings()
# dimention of the embedding vector
dimension=1536
neo4j_graph = Neo4jGraph() # instantiate the graph
create_constraints(neo4j_graph)
create_vector_index(neo4j_graph, dimension)

ValueError: Could not connect to Neo4j database. Please ensure that the url is correct